<a href="https://colab.research.google.com/github/ruchithareddy269/Dm-Assignment-3/blob/main/KDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.5 MB/s eta 0:

In [4]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **1. Data Selection**
First, we'll start by loading and exploring the uploaded dataset to understand its structure and content.

Please find the attached dataset-https://archive.ics.uci.edu/dataset/2/adult

In [14]:
diabetes_data = pd.read_csv('/content/drive/MyDrive/DM-Assignment-2-part2/diabetes_merged_date-time-sorted.csv')

# Display the first few rows of the dataset to get an overview
diabetes_data.head()

,datetime,code,value
0,1988-03-27 08:00:00,58,1
1,1988-03-27 08:00:00,34,0
2,1988-03-27 12:00:00,60,1
3,1988-03-27 18:00:00,62,2
4,1988-03-28 08:00:00,34,0


# **2. Data Preprocessing**

Check for missing values.
Check the data types of each column.
Explore basic statistics of the numerical columns.

In [15]:
# Check for missing values
missing_values = diabetes_data.isnull().sum()

# Check data types of each column
data_types = diabetes_data.dtypes

# Basic statistics for numerical columns
numerical_summary = diabetes_data.describe()

missing_values, data_types, numerical_summary

(datetime    0
 code        0
 value       0
 dtype: int64,
 datetime    object
 code         int64
 value        int64
 dtype: object,
                code         value
 count  29257.000000  29257.000000
 mean      46.493557      0.503948
 std       13.370339      0.798374
 min        4.000000      0.000000
 25%       33.000000      0.000000
 50%       48.000000      0.000000
 75%       60.000000      1.000000
 max       72.000000      4.000000)

# **3. Data Transformation**
Given that the datetime column is of object type, we'll convert it to a proper datetime format. Additionally, we can extract features like year, month, day, and hour for further analysis.

From our preprocessing:

Missing Values: There are no missing values in the dataset.
Data Types: The datetime column is of object type (likely string), while both code and value columns are integers.
Numerical Summary: The code column ranges from 4 to 72, and the value column ranges from 0 to 4.

In [16]:
# Convert 'datetime' column to datetime format
diabetes_data['datetime'] = pd.to_datetime(diabetes_data['datetime'])

# Extract year, month, day, and hour
diabetes_data['year'] = diabetes_data['datetime'].dt.year
diabetes_data['month'] = diabetes_data['datetime'].dt.month
diabetes_data['day'] = diabetes_data['datetime'].dt.day
diabetes_data['hour'] = diabetes_data['datetime'].dt.hour

# Display the transformed dataset
diabetes_data.head()

,datetime,code,value,year,month,day,hour
0,1988-03-27 08:00:00,58,1,1988,3,27,8
1,1988-03-27 08:00:00,34,0,1988,3,27,8
2,1988-03-27 12:00:00,60,1,1988,3,27,12
3,1988-03-27 18:00:00,62,2,1988,3,27,18
4,1988-03-28 08:00:00,34,0,1988,3,28,8


# **4. Data Mining**
For the purpose of this demonstration, let's make a simple prediction task: we'll try to predict the value based on the other features (excluding the original datetime column). Given that value is a categorical variable with a limited range, this becomes a classification problem.

We'll use a Random Forest classifier as our model. First, we need to split the data into training and testing sets. Let's proceed with this.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Splitting the data into training and testing sets
X = diabetes_data.drop(columns=['datetime', 'value'])
y = diabetes_data['value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train a Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

from sklearn.metrics import accuracy_score
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.7277853725222146

# **5. Interpretation/Evaluation**
While accuracy gives us a general idea of the model's performance, it's essential to delve deeper, especially with classification problems. Let's evaluate the model using additional metrics such as precision, recall, and the F1-score for each class in the value column.

In [19]:
from sklearn.metrics import classification_report

# Generate the classification report again
classification_rep = classification_report(y_test, y_pred)

classification_rep


'              precision    recall  f1-score   support\n\n           0       0.88      0.88      0.88      3830\n           1       0.51      0.55      0.53      1233\n           2       0.30      0.28      0.29       623\n           3       0.20      0.13      0.16       155\n           4       0.08      0.09      0.08        11\n\n    accuracy                           0.73      5852\n   macro avg       0.40      0.39      0.39      5852\nweighted avg       0.72      0.73      0.73      5852\n'